In [ ]:
# run this block, then start python dt-sentinel.py in shell. To stop all, just call wrapUp() 
cleanUp()
print "Num Backups:", int(open('NUM_BACKUPS.txt').readline())
server_list = {}
startSentinel()
time.sleep(1)
REDIS_SENTINEL = redis.StrictRedis(host='localhost', port=40001)
startServers(8)
sendMasterList()

In [ ]:
writeKKeys(100)

In [ ]:
cleanUp()

In [ ]:
for num_back in range(0, 9):
    t = []
    for i in range(10):
        print '#',i
        t.append (run(20, num_back, 10000))
        print "Avg: ", (sum(t))/(1.0 * len(t))

In [ ]:
import random
from random import randint
import subprocess
import os
import signal
from subprocess import call
import time
import string
import redis
import urllib
import urllib2
random.seed()

sentinelProc = None
server_list = {}
REDIS_SENTINEL = redis.StrictRedis(host='localhost', port=40001)

def getRandomString(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def getRandomPortNum():
    return randint(30000,40000)

def getUniqueRandomPortNum():
    p = randint(30000,40000)
    while True:
        unique = True
        for s in server_list:
            p2 = int(s.split(':')[1])
            if p2 == p:
                unique = False
                break
        if unique: return p
        p = randint(30000,40000)
    
def run(server_num, backup_num, key_count):
    cleanUp()
    print "Server Count:", server_num, "Num Backups:", setNumBackups(backup_num), "Key Count", key_count
    server_list.clear()
    startSentinel()
    time.sleep(1)
    REDIS_SENTINEL = redis.StrictRedis(host='localhost', port=40001)
    startServers(server_num)
    sendMasterList()
    time.sleep(1)
    t1 = time.time()
    writeKKeys(key_count)
    t2 = time.time()
    wrapUp()
    print "Time taken: ", str(t2 - t1)
    return t2-t1

def setNumBackups(N = 2):
    open('NUM_BACKUPS.txt', 'w').write(str(N))
    #print 'Num Backups', N
    return N
    
def getNumBackups():
    print 'Num Backups:' ,int(open('NUM_BACKUPS.txt').readline())
    
    
    

def startProc(args):
    return subprocess.Popen(args)

# p should be a process type
def killProc(proc):
    try:
        os.killpg(proc.pid, signal.SIGTERM)
#         call(["kill", "-9",str(proc.pid+1)]) #kill redis server pid+1 works for some reason
#         call(["kill", "-9", str(proc.pid)])
        return True
    except:
        print "Failed to kill proc pid:", proc.pid, " maybe doesn't exist"

def stopServer(addr=None,flag=True):
    if addr is None:
        addr = server_list.keys()[ randint(0,len(server_list.keys()) - 1)]
    p = addr.split(':')[1]
    s = "redis-cli -p " + str(p) + " shutdown nosave"
    # print "Killing redis server", addr
    call(s.split())
#     stopMonitoring(addr)
    if flag: server_list.pop(addr, None)
    killDtServer(addr)

def killDtServer(address=None):
    if address is None:
        address=server_list.keys()[randint(0,len(server_list.keys()) - 1)]
    try:
        url = '/shutDown'
        port = str((int)(address.split(':')[1])+20000)
        req = 'http://'+address.split(':')[0]+':'+port
        #print req+url
        response = urllib2.urlopen(req+url)
        return str(response.read())
    except Exception, e:
        print e
        return "Error shutting down", address
    
    
def killDtServer2(addr):
    p = int(addr.split(":")[1]) + 20000
    c = "lsof -t -i:"+str(p)
    task = subprocess.Popen(c.split(),stdout=subprocess.PIPE)
    pid = task.stdout.read().strip()
    if pid == "":
        print "No such proc:",addr
        return
    print "Killing DT server:", addr, "PID: ", pid
    call(["kill", pid])


def startDTSentinel():
    global sentinelProc
    s = 'python dt-sentinel.py'
    sentinelProc = startProc(s.split())
    
def stopDTSentinel():
    killProc(sentinelProc)
    

def killAllServers():
    for s in server_list:
        #print "Killing server:",s," pid:", server_list[s].pid
        #killProc(server_list[s])
        stopServer(s, False)
        
def startServer(ip = "127.0.0.1", port=None):
    if port == None:port=getRandomPortNum()
        
    s = ("redis-server --port " + str(port) + " --appendonly yes --appendfilename appendonly-" + str(port) + ".aof"
    " --dbfilename dump-" + str(port) + ".rdb --logfile " + str(port) + ".log --daemonize yes --loglevel verbose")
#     print s
#     print s.split()
    p = startProc(s.split())
    sp = ip + ":" + str(port)
    #print "Started server:",sp, " proc", p.pid
    server_list[sp] = p
    startMonitoring(sp)
    startDtServer(ip + ":" +str(port + 20000))
    time.sleep(1)

def startDtServer(addr):
    #print "Starting DT_Server", addr
    os.system("python dt_server.py "+addr.split(":")[0]+" "+ addr.split(":")[1]+ " > pdt_server_" + addr.split(":")[1] + ".txt 2>&1 &")

def startServers(numServers):
    server_list.clear()
    for i in range(numServers):
        startServer(port=getUniqueRandomPortNum())

        
def startMonitoring(addr):
    #print "masterP_" + addr
    REDIS_SENTINEL.sentinel_monitor("masterP_" + addr, addr.split(':')[0], addr.split(':')[1], 2)
def stopMonitoring(addr):
    REDIS_SENTINEL.sentinel_remove("masterP_" + addr)

def stopMonitoringAll():
    for i in server_list.keys(): 
        try:
            stopMonitoring(i)
        except:
            i = 0
def startSentinel():
    s = './create-multiredis/create-multiredis sentinel2'
    #call(s.split())
    startProc(s.split())
    #print 'Sentinel started at 40001'

def stopSentinel():
    #s = 'redis-cli -p 40001'
    s = './create-multiredis/create-multiredis stopsentinel'
    startProc(s.split())
    #print 'Sentinel stopped'

# def startDtSentinel():
#     s = "python dt-sentinel.py"
#     call(s.split())

def cleanUp():
    s = './create-multiredis/create-multiredis clean'
    call(s.split())

def push_to_server(cmd, args, addr=None):
    if addr == None:
        kk = randint(0,len(server_list.keys()) - 1)
        #print "rand:",kk, len(server_list.keys())
        addr = server_list.keys()[ randint(0,len(server_list.keys()) - 1)]
    data = urllib.urlencode(args)
    try:
        saddr = addr.split(":")[0] + ":" + str(int(addr.split(":")[1]) + 20000)
        url = "http://" + saddr + '/' + cmd + '?'
        response = urllib2.urlopen(url+data, timeout = 2)
        #print "POST Request:" + url+data
        result = response.read()
        #print "Response:" + result
        return result
    except:
        #print "No response"
        return "FAIL"

def sendListAsPost(url, slist):
    url = url + '/update_hash_to_back_map?'
#     print url, slist
    s = ''
    for i in slist: s = s + ',' + i
    s = s[1:]
    data = urllib.urlencode({'addrs' : s})
#     print 'encoded data', data
    try:
        response = urllib2.urlopen(url+data)
    #     print "POST Request:" + url+data
        result = response.read()
        return result
    except:
        return "Oops"

def getMastersList():
    return [REDIS_SENTINEL.sentinel_masters()[m]['ip'] + ':' + str(REDIS_SENTINEL.sentinel_masters()[m]['port']) for m in REDIS_SENTINEL.sentinel_masters() if not REDIS_SENTINEL.sentinel_masters()[m]['is_sdown']]

def sendMasterList():
    time.sleep(2)
    ml = getMastersList()
#     print "Master list:",ml
    #print "Updating masters map in dt servers"
    for url in ml:
        ip,port = url.split(':')
        sendListAsPost('http://'+ip+':'+str(int(port) + 20000), ml)
    #print "ok"

def get_key(key):
    return push_to_server("get",{"key":key})

def set_key(key, val):
    return push_to_server("set",{"key":key, "value":val})

def wrapUp():
    killAllServers()
    stopMonitoringAll()
    server_list.clear()
    stopSentinel()

def wrapUp2():
    killAllServers()
    server_list.clear()

#get and write the status to some log
def getStatus(address=None):
    if address is None:
        address=server_list.keys()[randint(0,len(server_list.keys()) - 1)]
    try:
        url = '/getKeyDist'
        port = str((int)(address.split(':')[1])+20000)
        req = 'http://'+address.split(':')[0]+':'+port
        print req+url
        response = urllib2.urlopen(req+url)
        return str(response.read())
    except Exception, e:
        print e
        return "Error retieveing status"

def getAllStatus():
    s = getStatus()[1:-1]
    l = s.split(',')
    for i in l: print i

def writeKKeys(r=1):
    # def test1(r=10):
    #r = 200
    #kl = []
    for i in range(r):
        rs = getRandomString()
        #kl.append(rs)
        set_key(rs,rs)
        #if int(((i*1.0)/r)*100) % 10 == 0: print int(((i*1.0)/r)*100)
    #     print set_key(rs,rs)
    #     print get_key(rs)
    #print "Wrote", r, "keys"

In [ ]:

    
def getRandomString():
    
    
# def startSentinel():

    
def killSentinel():
    

#start redis + DT server + notify sentinel
#update server_list
    

def startServers(numServers):

    

#kill the redis server at addr
#update server_list
def killServer(addr):
    

#get and write the status to some log
def getStatus():
    
#compute num migrations from two succesie status calls and write to file
def getNumMigrations():